In [1]:
import pandas as pd
import math

# DataFrame de exemplo

In [5]:
df = pd.DataFrame(data=[
    { 'expectativa': 'sol', 'temperatura': 'quente', 'humidade': 'alta', 'vento': 'fraco', 'jogar_tenis': 'nao' },
    { 'expectativa': 'sol', 'temperatura': 'quente', 'humidade': 'alta', 'vento': 'forte', 'jogar_tenis': 'nao' },
    { 'expectativa': 'nublado', 'temperatura': 'quente', 'humidade': 'alta', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'chuva', 'temperatura': 'moderada', 'humidade': 'alta', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'chuva', 'temperatura': 'fresco', 'humidade': 'normal', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'chuva', 'temperatura': 'fresco', 'humidade': 'normal', 'vento': 'forte', 'jogar_tenis': 'nao' },
    { 'expectativa': 'nublado', 'temperatura': 'fresco', 'humidade': 'normal', 'vento': 'forte', 'jogar_tenis': 'sim' },
    { 'expectativa': 'sol', 'temperatura': 'moderada', 'humidade': 'alta', 'vento': 'fraco', 'jogar_tenis': 'nao' },
    { 'expectativa': 'sol', 'temperatura': 'fresco', 'humidade': 'normal', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'chuva', 'temperatura': 'moderada', 'humidade': 'normal', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'sol', 'temperatura': 'moderada', 'humidade': 'normal', 'vento': 'forte', 'jogar_tenis': 'sim' },
    { 'expectativa': 'nublado', 'temperatura': 'moderada', 'humidade': 'alta', 'vento': 'forte', 'jogar_tenis': 'sim' },
    { 'expectativa': 'nublado', 'temperatura': 'quente', 'humidade': 'normal', 'vento': 'fraco', 'jogar_tenis': 'sim' },
    { 'expectativa': 'chuva', 'temperatura': 'moderada', 'humidade': 'alta', 'vento': 'forte', 'jogar_tenis': 'nao' }, 
])

In [6]:
df

,expectativa,temperatura,humidade,vento,jogar_tenis
0,sol,quente,alta,fraco,nao
1,sol,quente,alta,forte,nao
2,nublado,quente,alta,fraco,sim
3,chuva,moderada,alta,fraco,sim
4,chuva,fresco,normal,fraco,sim
5,chuva,fresco,normal,forte,nao
6,nublado,fresco,normal,forte,sim
7,sol,moderada,alta,fraco,nao
8,sol,fresco,normal,fraco,sim
9,chuva,moderada,normal,fraco,sim


# Arvore de decisão

Para decidir qual elemento utilizar como nó raiz, e também para escolher como serão feitas as quebras pode ser utilizado o indice de Gini ou o Ganho de informação

## Entropia

Medida que nos diz o quanto nossos dados estão desorganizados e misturados. Quanto maior a entropia, menor o ganho de informação e vice-versa. Nossos dados ficam menos entrópicos conforme dividimos os dados em conjuntos capazes de representar apenas uma classe do nosso modelo.

$H(x)=\displaystyle-\sum_{i=1}^n p(x_i)log_2p(x_i)$

A fórmula acima mostra que a entropia é a somatória das probabilidades de ocorrência de um elemento, multiplicada pelo logaritimo de base 2 da probabilidade de ocorrência.

Dando como exemplo, temos 14 registros onde 9 são SIM e 5 são NÃO,logo o cálculo de sua entropia seria:
$entropia=-(\frac{9}{14})*log_2(\frac{9}{14})-(\frac{5}{14})*log_2(\frac{5}{14})$<br>
$entropia=-0.64*-0.64-0.36*-1.47$<br>
$entropia=0.939$

In [8]:
df.groupby(['expectativa', 'jogar_tenis'])['expectativa'].count()

expectativa  jogar_tenis
chuva        nao            2
             sim            3
nublado      sim            4
sol          nao            3
             sim            2
Name: expectativa, dtype: int64

## Ganho de informação
O ganho de informação mede o quanto se ganha de pureza na seleção de determinado dado,quanto a maior a quantidade de impureza em um conjunto de dados, mais heterôgeneo ele será.

Para escolher qual propriedade deve ser utilizada no nó raiz, podemos pegar aquele que possuí o maior ganho

$Gain(S,A)=Entropia(S)-\sum_{v\in valores(A)}P(A_v)*Entropia(A_v)$<br>
Onde S é todo o conjunto, A é uma váriavel desse conjunto e v um valor de A, por exemplo Av pode ser igual a expectativa(A)=nublado(v).

$Entropia(sol)=-(\frac{3}{5})*log_2(\frac{3}{5})-(\frac{2}{5})*log_2(\frac{2}{5})=0.971$
$Entropia(nublado)=-(\frac{0}{4})*log_2(\frac{0}{4})-(\frac{4}{4})*log_2(\frac{4}{4})=0$
$Entropia(chuva)=-(\frac{2}{5})*log_2(\frac{2}{5})-(\frac{3}{5})*log_2(\frac{3}{5})=0.971$<br>
No nublado consideramos que o logaritimo de zero é zero e que 0 divido por qualquer valor será igual a zero.

$Gain(S,expectativa)=0.939-(\frac{5}{14}*0.971)-(\frac{4}{14}*0)-(\frac{5}{14}*0.971)$
$Gain(S,expectativa)=0.939-(0.357*0.971)-(0.286*0)-(0.357*0.971)$
$Gain(S,expectativa)=0.939-0.347-0-0.347=0.245$<br>

Nossa propriedade expectativa possui um ganho de informação de 0.245 bits

In [59]:
# quantidade_total Quantidade de vezes que o valor aparece no conjunto
# quantidade_por_classe Quantidade de vezes que o valor aparece em cada classe(5 vezes no sim e 8 no não por exemplo)
def calc_entropia(quantidade_total, quantidade_por_classe):
    entropia = None
    for valor in quantidade_por_classe:
        if valor == 0:
            continue
        if entropia == None:
            entropia = -(valor/quantidade_total)*math.log2(valor/quantidade_total)
            continue
        entropia -= (valor/quantidade_total)*math.log2(valor/quantidade_total)
    
    return entropia

In [101]:
def pega_qtd_vezes_valores_aparecem(conjunto, classe_alvo, coluna_condicional=None, valor_alvo=None):
    if coluna_condicional == None:
        return conjunto.groupby(classe_alvo)[classe_alvo].count().tolist()
    return conjunto[conjunto[coluna_condicional] == valor_alvo].groupby(classe_alvo)[classe_alvo].count().tolist()
    
def calc_ganho_informacao(conjunto, classe_alvo):
    tamanho_conjunto = conjunto.shape[0]
    valores_classe_alvo = conjunto[classe_alvo].unique().tolist()
    
    entropia_conjunto = calc_entropia(
        tamanho_conjunto,
        pega_qtd_vezes_valores_aparecem(conjunto, classe_alvo)
    )
    
    ganho_informacao_variaveis = dict()
    for coluna in conjunto.columns:
        entropias_coluna = []
        
        if coluna == classe_alvo:
            continue
        
        for valor in conjunto[coluna].unique().tolist():
            total_aparicao_valor = conjunto[conjunto[coluna] == valor][coluna].count()
            qtd_aparicao_valor_por_classe = pega_qtd_vezes_valores_aparecem(
                conjunto,
                classe_alvo,
                coluna,
                valor
            )
            entropia_valor = calc_entropia(total_aparicao_valor, qtd_aparicao_valor_por_classe)
            entropias_coluna.append((entropia_valor,total_aparicao_valor))
            
        entropia_media_coluna = 0
        for ent, qtd_valor in entropias_coluna:
            entropia_media_coluna += ((qtd_valor/tamanho_conjunto)*ent)
        
        ganho_informacao_variaveis[coluna] = entropia_conjunto - entropia_media_coluna
        
    return ganho_informacao_variaveis

In [102]:
calc_ganho_informacao(df, 'jogar_tenis')

{'expectativa': 0.24674981977443933,
 'temperatura': 0.02922256565895487,
 'humidade': 0.15183550136234159,
 'vento': 0.04812703040826949}

## Escolha da variável
A expectativa deve ser nosso nó raiz, pois ela possui o maior ganho de informação

# Indice GINI
Usado para medir a frêquencia que um elemento será escolhido erroneamente, quanto menor o indice melhor a escolha

$Gini(x)=1-\sum_jP^2_j$

            -----------------------------
            |  Pessoas:           15328 |
            |  Inadimplente(SIM): 542   |
            |  Inadimplente(NAO): 14786 | 
            -----------------------------
    Possui imóvel? Sim      |   Possui imóvel? Não
       ------------------------------------------------------          
       |                                                    |
    -----------------------------       ----------------------------- 
    |  Pessoas:           12779 |       |  Pessoas:           2549  |
    |  Inadimplente(SIM): 440   |       |  Inadimplente(SIM): 102   |
    |  Inadimplente(NAO): 12339 |       |  Inadimplente(NAO): 2447  |
    -----------------------------       -----------------------------
    
Com base na arvóre acima vamos calcular os indices de Gini

###### Possui imóvel
$Gini(ImovelS)= 1-[(\frac{440}{12779})^2+(\frac{12339}{12779})^2]=0.07$<br>
$Gini(ImovelN)= 1-[(\frac{102}{2549})^2+(\frac{2447}{2549})^2]=0.08$<br>

Logo a impureza de gini do nosso nó raiz será<br>
$Impureza=[(\frac{12779}{15328})*0.07]+[(\frac{2549}{15328})*0.08]=0.07$

In [12]:
def calc_gini(total, valores):
    acm = 0
    for valor in valores:
        acm += (valor/total)**2
    return round(1-acm, 2)

def calc_impureza_gini(total, valores_gini):
    acm = 0
    for valor, gini in valores_gini:
        acm += (valor/total)*gini
    return round(acm, 2)

In [13]:
calc_impureza_gini(
    15328, 
    [
        (12779, calc_gini(12779, [440,12339])),
        (2549, calc_gini(2549, [102,2447]))
    ]
)

0.07